## prod and slicing in gurobipy and ticdat

I'm playing around with adding a `prod` to `ticdat`. It will only work with the `gurobipy.tupledict` object, so this is an engine biased function.

What's interesting is that because you define the field names for `ticdat`, you can slice by name instead of by position. This really improves readability in my opinion. 

Have you guys ever discussed named slicing? It would probably be an easy extention to `gurobipy.tupledict` to include field (aka position) names when calling `addVars`.

In [1]:
import gurobipy as gu
from ticdat import TicDatFactory

In [2]:
mdl = gu.Model()
temp_vars = mdl.addVars([(1,2), ("a","b")], vtype=gu.GRB.BINARY,name='temp_vars')
temp_vars

{(1, 2): <gurobi.Var temp_vars[1,2]>, ('a', 'b'): <gurobi.Var temp_vars[a,b]>}

In [3]:
tdf = TicDatFactory(table = [["Field One", "Field Two"],["Data Field"]])
dat = tdf.TicDat(table = {(1,2):12, ("a","b"):2})
dat.table

{(1, 2): _td:{'Data Field': 12}, ('a', 'b'): _td:{'Data Field': 2}}

Here is positional slicing with `gurobipy.prod`. For `ticdat` to play along, it needs collapse its inner dicts to actual data values with the `field_dict` function.

In [4]:
temp_vars.prod(dat.table.field_dict(), 'a', '*')

<gurobi.LinExpr: 2.0 temp_vars[a,b]>

Here is named slicing. It renders the same result, but it's more self-documenting. It's more explicit than positional slicing in indicating that the product is over those tuples whose "field_one" value is 'a'.

In [5]:
dat.table.prod(temp_vars, field_one = "a")

<gurobi.LinExpr: 2.0 temp_vars[a,b]>

Let's see it again - first positional slicing.

In [6]:
temp_vars.prod(dat.table.field_dict(), '*', 2)

<gurobi.LinExpr: 12.0 temp_vars[1,2]>

Then named slicing.

In [7]:
dat.table.prod(temp_vars, field_two = 2)

<gurobi.LinExpr: 12.0 temp_vars[1,2]>

Just to include the basic case, we verify that both `ticdat` and `gurobipy` implement the same `prod`.

In [8]:
temp_vars.prod(dat.table.field_dict())

<gurobi.LinExpr: 12.0 temp_vars[1,2] + 2.0 temp_vars[a,b]>

In [9]:
dat.table.prod(temp_vars)

<gurobi.LinExpr: 12.0 temp_vars[1,2] + 2.0 temp_vars[a,b]>